# Representación de palabras mediante espacios vectoriales

## Créditos

Estas clases y material están fuertemente inspirado en los cursos de Stanford [CS224n: Natural Language Processing with Deep Learning](http://web.stanford.edu/class/cs224n/) y [CS224u: Natural Language Understanding](http://web.stanford.edu/class/cs224u/).

In [ ]:
__author__ = "Cristian Cardellino"

## Contenidos

1. [Representación de palabras](#Representación-de-palabras)
1. [Representación distribuída de palabras](#Representación-distribuída-de-palabras)
1. [Diseño de matrices](#Diseño-de-matrices)
1. [Comparación de vectores](#Comparación-de-vectores)
1. [Métodos de reponderación](#Métodos-de-reponderación)

In [ ]:
%matplotlib inline

import numpy as np
import os
import pandas as pd
import vsm

## Representación de palabras

1. [Características de las palabras](#Características-de-las-palabras)
1. [Representación mediante taxonomías](#Representación-mediante-taxonomías)
1. [Representación discreta](#Representación-discreta)
1. [Ejemplo de representación discreta](#Ejemplo-de-representación-discreta)

### Características de las palabras

- Son la **unidad básica** en cualquier tarea de procesamiento de lenguaje natural (PLN).
- Cualquier modelo de PLN requiere como entrada la **representación de una palabra**.
- Gran parte del trabajo de PLN solía representar las palabras como **símbolos atómicos**.
- Las nociones de **similitu y distancia** entre palabras son cruciales para tareas de PLN.

### Representación mediante taxonomías

- El **sentido** de una palabras es la idea que una palabra (o frase) representa.
- Se puede representar mediante recursos del estilo de las **taxonomías**.
- Ofrecen **información rica y estructurada**.
    - WordNet asigna relaciones de **hiperonimia/hiponimia** (relación *es-un*), y **sinonimia**.
    - FrameNet establece la **estructura** mediante un marco sintáctico y semántico de las palabras.
- Son **difíciles y caros de obtener y actualizar** (requiren anotación humana).

In [ ]:
from nltk.corpus import wordnet as wn  # Run nltk.download("wordnet") first

dog = wn.synset("dog.n.01")

print("Dog's hypernyms:")
for hypernym in dog.hypernyms():
    print(hypernym)

### Representación discreta

- Para aplicar medidas de similitud y distancia (e.g. euclídea, coseno), utilizamos **vectores que representen palabras**.
- Primera aproximación: **vectores *one-hot***.
- Para cada palabra del vocabulario, tenemos un vector.
- En términos de espacio vectorial, el vector tiene un **1** en una de sus dimensiones y __0s__ en todas las demás.
- **Dimensionalidad muy alta** (el inglés tiene un estimado de 13 millones de palabras).
- Los vectores son **ralos** (o esparsos), tienen muchos ceros.

### Ejemplo de representación discreta

Dado un corpus:

- "El fallo fue decisivo"
- "La corte rectificará el fallo"

Tenemos un vocabulario $V = \{corte, decisivo, el, fallo, fue, la, rectificará\}$.

Codificamos un número $|V| = 7$ de vectores para cada palabra $w^{(i)} \in \mathbb{R}^{|V|}$.

### Ejemplo de representación discreta

$$
    w^{corte} =
    \begin{bmatrix}
       1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{decisivo} =
    \begin{bmatrix}
       0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{el} =
    \begin{bmatrix}
       0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{fallo} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
$$

$$
    w^{fue} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
    \end{bmatrix}
    w^{la} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0
    \end{bmatrix} ,
    w^{rectificará} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1
    \end{bmatrix}
$$

In [ ]:
corpus = [
    "El fallo fue decisivo", 
    "La corte rectificará el fallo"
]

tokenized_corpus = [
    [word.lower() for word in doc.split()] for doc in corpus
]

vocabulary = sorted(set([
    word for doc in tokenized_corpus for word in doc
]))

one_hot_vectors = np.eye(len(vocabulary), dtype=np.int)

for doc in tokenized_corpus:
    print(np.array([
        one_hot_vectors[vocabulary.index(word), :] for word in doc
    ]))

## Representación distribuída de palabras

1. [Concepto](#Concepto)
1. [Motivación](#Motivación)
1. [Hipótesis para guiarse](#Hipótesis-para-guiarse)

### Concepto

- Idea: Representar una palabra **mediante sus vecinos**.
    - Es una de las **ideas más importantes** en PLN moderno.

#### Ejemplo: 

- "_El gobierno rescató a los_ **bancos** _ante la crisis de crédito_".
- "_La municipalidad reparará los_ **bancos** _de la plaza_".

En estos ejemplos, las palabras que están en _cursiva_ representarán las palabras en **negrita**.

### Motivación

- La representación distribuída nos da pie a pensar como los vectores **codifican el significado de las unidades linguísticas** (palabras, oraciones, documentos, etc.).
- Son la base de los **modelos de espacio vectorial**.
- Piezas fundamentales en modelos para **Procesamiento y Comprensión de Lenguaje Natural**.
- Estas representaciones pueden ser usadas, entre otros, para:
    - Entender y modelar fenómenos sociales y lingüísticos.
    - Entrada para modelos de aprendizaje automático.

### Hipótesis para guiarse

- "You shall know a word by the company it keeps" (Firth, 1957).
- "The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously" (Firth, 1957).
- "The meaning of a word is its use in the language" (Wittgenstein, 1957).
- "Distributional statements can cover all of the material of a language without requiring support from other types of information" (Harris, 1954)
- "If units of text have similar vectors in a text frequency matrix, then they tend to have similar meanings." (Turney & Pantel, 2010)

## Diseño de matrices

1. [Objetivos del diseño de matrices](#Objetivos-del-diseño-de-matrices)
1. [Matriz de co-ocurrencia de palabras](#Matriz-de-co-ocurrencia-de-palabras)
1. [Matriz de documentos](#Matriz-de-documentos)
1. [Ventana y ponderación](#Ventana-y-ponderación)
1. [Más diseños de matrices](#Más-diseños-de-matrices)
1. [Consideraciones](#Consideraciones)

### Objetivos del diseño de matrices

- Buscamos representar palabras de manera matemática.
    - Idea: Mediante una matriz (vista como un conjunto de vectores)
- Hay que definir la construcción de una matriz.
    - Muchas opciones de diseño.
    - Distinto impacto tendrán las decisiones tomadas respecto al modelado del texto.

### Matriz de co-ocurrencia de palabras

- Cada palabra se representa por las palabras a su alrededor (dada una ventana).
- La matriz es simétrica (independiente de la posición de las palabras).
    - Esto se conoce como "bolsa de palabras" (o _bag of words_)
- Dado un vocabulario $V$:
    - Matriz de co-ocurrencias $X \in \mathbb{R}^{|V|\times|V|}$
    - $X_{ij}$ representa la co-ocurrencia entre la palabras $w^{(i)}$ y $w^{(j)}$

### Matriz de co-ocurrencia de palabras

<img src="img/word-word-matrix.png" width="90%"/>
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Matriz de documentos

- Cada documento se representa por las palabras que lo conforman.
- Se usa un modelo de bolsa de palabras: no importa la posición.
- Dadas $N$ palabras en un corpus de $M$ documentos:
    - Matriz de documentos $X \in \mathbb{R}^{N \times M}$
    - $X_{ij}$ representa la ocurrencia de la palabra $w^{(i)}$ en el documento $d^{(j)}$.

### Matriz de documentos

<img src="img/term-document-matrix.png" width="90%"/>
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Ventana y ponderación

- La ventana define la **cantidad de palabras en el contexto** a considerar.
- La ponderación (*scaling*) le da un **peso a la palabra** en la suma total.
    - Igual peso para todas las palabras, i.e. bolsa de palabras.
    - Peso distinto de acuerdo a la distancia, e.g. decaimiento fraccional.
    - Peso binario, i.e. la palabra está o no.

<img src="img/window-scaling.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Ventana y ponderación

- Ventanas más grandes y con peso igual (_flat_) capturan **información semántica**.
    - Las matrices de documentos dan idea de la temática del documento.
- Ventanas más cortas y con pesos capturan **información sintáctica** (colocacional).
    - Las matrices de co-ocurrencias de palabras son de afinidad de palabras.
- El límite puede definirse más allá de una ventana fija.
    - Utilizar oraciones, párrafos o documentos tendrá consecuencias distintas.

In [ ]:
infoleg5 = pd.read_csv("./data/infoleg_5window_scaled.csv.gz", index_col=0)

infoleg20 = pd.read_csv("./data/infoleg_20window_flat.csv.gz", index_col=0)

### Más diseños de matrices

- palabra $\times$ relación de dependencia
- palabra $\times$ contexto sintáctico
- adjetivo $\times$ sustantivo modificado
- palabra $\times$ consulta de búsqueda
- persona $\times$ producto
- palabra $\times$ persona
- verbo $\times$ sujeto $\times$ objeto
- ...

### Consideraciones

- Las matrices **incrementan con el vocabulario**.
    - Definir límites en vocabulario o bien lidiar con modelos muy grandes.
    - Por la ley de Zipf (Zipf, 1949), la lista de palabras con escasa aparición se vuelve muy grande.
- La elección de la ventana afecta en la obtención de **matrices más o menos dispersas**.
- Representar distintas unidades lingüísticas deriva en **más dispersión** (ver el ejemplo de word-word vs term-document).

## Comparación de vectores 

1. [Objetivos de comparar vectores](#Objetivos-de-comparar-vectores)
1. [Ejemplo de juguete](#Ejemplo-de-juguete)
1. [Distancia euclídea](#Distancia-euclídea)
1. [Normalización del vector](#Normalización-del-vector)
1. [Distancia coseno](#Distancia-coseno)
1. [Medidas basadas en coincidencias](#Medidas-basadas-en-coincidencias)
1. [Propiedades de una distancia](#Propiedades-de-una-distancia)
1. [Otras medidas](#Otras-medidas)
1. [Exploración de vecinos](#Exploración-de-vecinos)

### Objetivos de comparar vectores

- Está en el núcleo de nuestro análisis.
- Generalmente buscamos **medir distancias** entre vectores.
    - La idea es que palabras relacionadas estén cercanas en el espacio vectorial construído.
- El módulo [scipy.spatial.distance](http://docs.scipy.org/doc/scipy-0.14.0/reference/spatial.distance.html) ofrece una gran cantidad de métricas para comparar vectores.

### Ejemplo de juguete

<img src="img/vector-toy-example.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Ejemplo de juguete

In [ ]:
ABC = pd.DataFrame([
    [ 2.0,  4.0], 
    [10.0, 15.0], 
    [14.0, 10.0]],
    index=['A', 'B', 'C'],
    columns=['dx', 'dy'])

ABC

### Ejemplo de juguete

In [ ]:
def plot_ABC(df):
    ax = df.plot.scatter(x='dx', y='dy', marker='.', legend=False)
    m = df.values.max(axis=None)
    ax.set_xlim([0, m*1.2])
    ax.set_ylim([0, m*1.2])
    for label, row in df.iterrows():
        ax.text(row['dx'], row['dy'], label)

plot_ABC(ABC)

### Distancia euclídea

- Es la **distancia** más básica e intuitiva entre dos vectores.
- La distancia euclídea entre dos vectores $u$ y $v$ de dimensión $n$ se define como:

$$\textbf{euclidean}(u, v) = \sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

- En dos dimensiones, se corresponde con la longitud de la línea más directa entre dos puntos.

<img src="img/euclidean-distance.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Distancia euclídea

In [ ]:
def abc_comparisons(df, distfunc):
    for a, b in (('A', 'B'), ('B', 'C')):
        dist = distfunc(df.loc[a], df.loc[b])
        print(f"{distfunc.__name__}({a}, {b}) = {dist:7.02f}")

abc_comparisons(ABC, vsm.euclidean)

- Suponiendo que estos vectores fueran de palabras, hay algunos aspectos extraños:
    - Notar que las distribuciones de B y C son opuestas (e.g. "bueno" vs. "malo").
    - A y B están más alineadas en cambio, salvo por las frecuencias (e.g. "bueno" vs. "excelente").
- La distancia euclídea está **sesgada por tamaño**.

### Normalización del vector

- Buscamos quitar el sesgo. Para eso **normalizamos el vector de acuerdo a su longitud**.
- Definimos la **norma L2** de un vector:

$$\|u\|_{2} = \sqrt{\sum_{i=1}^{n} u_{i}^{2}}$$

- Luego, la normalización de $u$ se consigue dividiendo por su longitud (que es escalar):

$$\left[\frac{u_{1}}{\|u\|_{2}}, \frac{u_{2}}{\|u\|_{2}}, \ldots, \frac{u_{n}}{\|u\|_{2}}\right]$$

### Normalización del vector

<img src="img/length-normalization.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Normalización del vector

In [ ]:
ABC_normed = ABC.apply(vsm.length_norm, axis=1)
plot_ABC(ABC_normed)

In [ ]:
abc_comparisons(ABC_normed, vsm.euclidean)

### Distancia coseno

- La **distancia coseno toma toda la longitud en cuenta**.
- La distancia coseno entre dos vectores $u$ y $v$ de dimensión $n$ se define como:

$$\textbf{cosine}(u, v) = 1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

- La similitud coseno (el término que se está restando), **mide el ángulo entre dos vectores**.
- Es **equivalente** a tomar la distancia euclídea de los vectores normalizados.

In [ ]:
abc_comparisons(ABC, vsm.cosine)

### Distancia coseno

<img src="img/cosine-distance-1.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Distancia coseno

<img src="img/cosine-distance-2.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Medidas basadas en coincidencias

- El método básico basado en coincidencia crea un vector que mide la cantidad de coincidencias entre dos vectores:

$$\textbf{matching}(u, v) = \sum_{i=1}^{n} \min(u_{i}, v_{i})$$

- El [__coeficiente de Jaccard__](https://en.wikipedia.org/wiki/Jaccard_index) es una manera de normalizar el método de coincidencias.

$$\textbf{jaccard}(u, v) = 1 - \frac{\textbf{matching}(u, v)}{\sum_{i=1}^{n} \max(u_{i}, v_{i})}$$

In [ ]:
abc_comparisons(ABC, vsm.jaccard)

### Propiedades de una distancia

Para que una medida califique como distancia, debe cumplir algunos requisitos.

- Simétrica: $d(u, v) = d(v, u)$.
- Asignar 0 a vectores idénticos: $d(v, v) = 0$
- Satisfacer la **desigualdad triangular**: $d(u, w) \leq d(u,v) + d(v, w)$
    - La distancia coseno, definida arriba, no satisface esta última (hay otras maneras de definirla).

### Otras medidas

Muchas otras medidas existen con ligeras variaciones. La mayoría se encuentran implementadas en [scipy](https://www.scipy.org/).

- [Coeficiente de Dice](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)
- [Coeficiente de superposición](https://en.wikipedia.org/wiki/Overlap_coefficient)
- [KL Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)
    - Symmetric KL
    - KL-divergence with Skew
    - [Jensen-Shannon](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence).

De estas, sólo Jensen-Sahnnon es una métrica de distancia.

### Exploración de vecinos

- La función `vsm.neighbors` es útil para investigación.
- Para una palabra dada `w`, hace un ranking de todas las palabras en el vocabulario de acuerdo a su distancia con `w`, medida por `distfunc`.

In [ ]:
vsm.neighbors("senado", infoleg5, distfunc=vsm.cosine).head()

In [ ]:
vsm.neighbors("senado", infoleg5, distfunc=vsm.euclidean).head()

### Exploración de vecinos

In [ ]:
vsm.neighbors("senado", infoleg20, distfunc=vsm.cosine).head()

In [ ]:
vsm.neighbors("senado", infoleg20, distfunc=vsm.euclidean).head()

## Métodos de reponderación

1. [Objetivos de reponderar](#Objetivos-de-reponderar)
1. [Normalización de conteo](#Normalización-de-conteo)
1. [Observado/Esperado](#Observado/Esperado)
1. [Información mutua punto a punto](#Información-mutua-punto-a-punto)
1. [TF-IDF](#TF-IDF)
1. [Distribución de valores de acuerdo a esquemas de reponderación](#Distribución-de-valores-de-acuerdo-a-esquemas-de-reponderación)
1. [Observaciones respecto a reponderación](#Observaciones-respecto-a-reponderación)

### Objetivos de reponderar

- Amplificar lo importante, lo confiable o lo inusual. Minimizar lo mundano o lo común.
- La idea de moverse del conteo plano es que las frecuencias son una aproximación pobre.
- Sobre cada esquema de reponderación hay que preguntarse:
    - ¿Cómo se compara con valores de conteo sin procesar?
    - ¿Cómo se compara con la frecuencia de palabras?
    - ¿Qué distribución general de valores devuelve?
- Se buscan métodos que no requieran intervención extra (e.g. remoción de palabras vacías, selección de atributos, etc.)

### Normalización de conteo

- Es la forma más básica para hacer reponderación.
- Se vió anteriormente como realizar [normalización mediante longitud L2](#Normalización-del-vector).
- Se puede normalizar además, mediante la suma de sus valores, y obtener una distribución de probabilidad:

$$\left[ \frac{u_{1}}{\sum_{i=1}^{n}u_{i}}, \frac{u_{2}}{\sum_{i=1}^{n}u_{i}}, \ldots, \frac{u_{n}}{\sum_{i=1}^{n}u_{i}} \right]$$

- Estas normalizaciones son **insensibles a cambios en la magnitud**, algo que puede ser perjudicial cuando tenemos muchos datos:
    - Los vectores $[1, 10]$ y $[1000,10000]$ son muy distintos, pero la normalización lo oscurecerá.

### Observado/Esperado

- El cálculo del valor esperado define una celda si las dos palabras fueran independientes. 
- Al utilizar el valor observado y dividirlo por el esperado, se amplifican aquellos valores cuya observación supera a la esperanza.


$$\textbf{rowsum}(X, i) = \sum_{j=1}^{n}X_{ij}$$

$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$

$$\textbf{oe}(X, i, j) = \frac{X_{ij}}{\textbf{expected}(X, i, j)}$$

### Observado/Esperado

<img src="img/observed-expected-1.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Observado/Esperado

<img src="img/observed-expected-2.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Observado/Esperado

In [ ]:
infoleg5_oe = vsm.observed_over_expected(infoleg5)
vsm.neighbors("senado", infoleg5_oe).head()

In [ ]:
infoleg20_oe = vsm.observed_over_expected(infoleg20)
vsm.neighbors("senado", infoleg20_oe).head()

### Información mutua punto a punto

- La información mutua punto a punto (*PMI* en inglés), es observado/esperado en espacio logarítmico:

$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

- Tiene un problema para co-ocurrencias nulas. Si definimos $\log(0) = 0$ tenemos un problema:
    - Observado > Esperado $\Rightarrow$ PMI alta.
    - Observado < Esperado $\Rightarrow$ PMI baja.
    - Conteo $0 \Rightarrow$ En el medio de las anteriores.
    
- La forma de lidiar con esto: utilizar **PMI Positiva**:

$$\textbf{ppmi}(X, i, j) = \max(0, \textbf{pmi}(X, i, j))$$

### Información mutua punto a punto

<img src="img/pmi.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Información mutua punto a punto

In [ ]:
infoleg5_ppmi = vsm.pmi(infoleg5, positive=True)
vsm.neighbors("senado", infoleg5_ppmi).head()

In [ ]:
infoleg20_ppmi = vsm.pmi(infoleg20, positive=True)
vsm.neighbors("senado", infoleg20_ppmi).head()

### TF-IDF

- Uno de los esquemas de reponderación más conocidos: **Term Frequency–Inverse Document Frequency (TF-IDF)** (literalmente: _Frecuencia de Palabra–Frecuencia Inversa de Documento_). Muy utilizado por los buscadores web.
- Está definido en base a las medidas de *TF* e *IDF*. Para una matriz de $m$ términos y $n$ documentos, se definen:

$$\textbf{TF}(X, i, j) = \frac{X_{ij}}{\textbf{colsum}(X, i, j)}$$

$$\textbf{IDF}(X, i, j) = \log\left(\frac{n}{|\{k : X_{ik} > 0\}|}\right)$$

$$\textbf{TF-IDF}(X, i, j) = \textbf{TF}(X, i, j) \cdot \textbf{IDF}(X, i, j)$$

### TF-IDF

<img src="img/tfidf.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### TF-IDF

- TF-IDF suele funcionar mejor con matrices ralas.
- Está pensado más bien para una matriz de documentos, en lugar de una matriz de co-ocurrencia de palabras.
- En particular, no funcionaría bien para nuestros datos de InfoLEG.
    - Si modelamos una matriz de documentos (leyes, artículos, resoluciones, etc.) y palabras, TF-IDF tiene más sentido.

__Importante__: Notar que [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer), de `scikit-learn` asume que la matriz de documentos utiliza los documentos como fila y las palabras como columna (serían los _atributos_ del modelo). Esto tiene más sentido para un trabajo de clasificación. El paquete `vsm` posee una implementación bastante sencilla de TF-IDF para documentos en forma de columna.

In [ ]:
# Equivalente a CountVectorizer + TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 

vectorizer = TfidfVectorizer("filename", max_features=5000)
corpus = sorted(os.path.join("./data/infoleg_text", fname) for fname in os.listdir("./data/infoleg_text/"))
vectorized_corpus = vectorizer.fit_transform(corpus)

### Distribución de valores de acuerdo a esquemas de reponderación

<img src="img/weighting-scheme-distribution.png" width="75%" />
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Observaciones respecto a reponderación

- Se pesa el valor de una celda $X_{ij}$ respecto al valor esperado que dan $X_{i*}$ y $X_{*j}$.
- Algunos esquemas terinan favoreciendo eventos muy raros (ver PMI). Esto puede derivar en amplificación de ruido, algo bastante común cuando lidiamos con datos de lenguaje.
- La magnitud puede ser importante, por lo que esquemas de normalización llana pueden oscurecer información valiosa.
- TF-IDF castiga severamente las palabras que aparecen en muchos documentos, por lo que no funciona bien con un esquema denso como el de matrices de co-ocurrencias entre palabras.

## Referencias

- Firth, J. R. (1957). A synopsis of linguistic theory, 1930-1955. Studies in linguistic analysis.
- Harris, Z. S. (1954). Distributional structure. Word, 10(2-3), 146-162.
- Turney, P. D., & Pantel, P. (2010). From frequency to meaning: Vector space models of semantics. Journal of artificial intelligence research, 37, 141-188.
- Wittgenstein, L. (1953). Philosophical investigations. (Translated by GE Anscombe) Oxford: Blackwell.
- Zipf, G. K. (1949). Human behavior and the principle of least effort.